In [2]:
import json
import pandas as pd
import re

In [52]:
path = '/Users/bleopold/OneDrive/data-analysis/vendors/WikiSQL/data/dev.tables.jsonl'
base_target_path = "../data/01_raw/wiki-sql"

In [54]:
column_names = ['table_id', 'column_name_original', 'column_name_clean', 'column_order']
mapping_info_all_tables = pd.DataFrame(
    columns = column_names
)


with open(path, 'r') as json_file:
    tables_info_raw = list(json_file)
    
for table in tables_info_raw:
    result = json.loads(table)
    
    table_id  = get_table_id(result)
    original_column_names = get_table_columns(result)
    
    
    mapping_info_one_table = pd.DataFrame(
        columns = column_names
    )

    for idx, column_name_original in enumerate(original_column_names):
        column_name_clean = slugify(column_name_original)


        mapping_info_one_table.loc[idx, "column_name_original"] = column_name_original
        mapping_info_one_table.loc[idx, "column_name_clean"] = column_name_clean
        mapping_info_one_table.loc[idx, "column_order"] = idx
        mapping_info_one_table[["table_id"]] = table_id

    mapping_info_all_tables = mapping_info_all_tables.append(mapping_info_one_table)

    mapping_info_all_tables.reset_index(drop=True, inplace=True)
    
mapping_info_all_tables.to_csv(
        "{}/mapping_column_names.csv".format(base_target_path)
        , sep='\t'
        , index=False
    )

In [31]:
def slugify(value):
    """
    Normalize string name to be able to save to disk
    
    Based on: https://stackoverflow.com/questions/295135/turn-a-string-into-a-valid-filename
    """
    value = re.sub('[^\w\s-]', '', value).strip().lower()
    value = re.sub('[^\w\s-]', '', value).strip().lower()
    value = value.replace(' ', '_')
    return value

In [17]:
def get_table_columns(result):
    table_columns = result['header']
    return table_columns

In [18]:
def get_table_id(result):
    table_id = result["id"]
    return table_id

In [5]:
def get_data_for_table_as_df(result):
    table_columns = result['header']
    table_columns = [slugify(_) for _ in table_columns]
    
    table_rows = result['rows']

    table_df = pd.DataFrame(columns = table_columns)
    
    for table_row in table_rows:
        table_row_df = pd.DataFrame(columns = table_columns, data=[table_row])
        table_df = table_df.append(table_row_df)
    
    table_df.reset_index(drop=True, inplace=True)
    return table_df

In [6]:
columns_mapping_df = ["table_id", "table_name"]
df_mapping = pd.DataFrame(columns=columns_mapping_df)


with open(path, 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list[0:10]:
    result = json.loads(json_str)
    
    # get table only if page_title exists
    try:
        page_title = result["page_title"]
        page_title = slugify(page_title)
    except KeyError as e:
        continue
        
        
    table = get_data_for_table_as_df(result)
    
    # save table to disk
    table_id = get_table_id(result)
    table_file_path = '{}/{}.csv'.format(base_target_path, table_id)
    table.to_csv(table_file_path, sep='\t', index=False)
    
    # store mapping between table_id and table_name
    df_mapping_ = pd.DataFrame(columns=columns_mapping_df, data=[[table_id, page_title]])
    df_mapping = df_mapping.append(df_mapping_)
    df_mapping.to_csv(
        "{}/mapping_table_id_page_title.csv".format(base_target_path)
        , sep='\t'
        , index=False
    )